In [1]:
# Importing Libraries
import selenium
import pandas as pd
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException
import warnings
warnings.filterwarnings('ignore')

### SCRAPING DATA OF REVIEWS FROM AMAZON

In [3]:
# Creating python program to search desired keywords one by one and saving links in open list prod_url

# creating open list to save href links of all products 
prod_url = []

# Keywords for search to which we need to scrape the data.
keywords = ['Laptops', 'Phones', 'Headphones', 'Smart Watches', 'Cameras', 'Printers', 'Monitors','Home theater', 'Router']

# Location of chrome web driver and launching it
driver = webdriver.Chrome("chromedriver.exe") 

# website from which we will scrape data
driver.get("https://www.amazon.in/") 

initial_url = driver.current_url

# loop for sending keywords to website and scraping href
for i in keywords:
    # Finding search input and giving input
    search = driver.find_element_by_id("twotabsearchtextbox").send_keys(i)
        
    # clicking on search button
    search_btn = driver.find_element_by_xpath("//div[@class='nav-search-submit nav-sprite']")
    search_btn.click()
    # giving time to load the webpage completely to avoid errors
    time.sleep(5)
    # finding href of each products mentioned in keywords
    for j in driver.find_elements_by_xpath("//h2[@class = 'a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a"):
        prod_url.append(j.get_attribute('href'))
    # returning to initial url    
    driver.get(initial_url)

In [4]:
# checking length of prod_url
len(prod_url)

233

In [5]:
# opening each 226 prod_url and scraping links of all reviews href 
# open list to save all reviews href
rev_cli = []

# loop for opening all links stored in prod_url
for j in prod_url:
    # opening each links of prod_url one by one
    driver.get(j)
    
    try:
        # finding links from all webpage by xpath
        for k in driver.find_elements_by_xpath("//a[@id = 'acrCustomerReviewLink']"):
            rev_cli.append(k.get_attribute('href'))
    # if all reviews link is not available, it will continue to next page,so avoiding NoSuchElementException 
    except NoSuchElementException as e:
        continue

In [6]:
# Checking length of open list
len(rev_cli)

455

In [7]:
# Now opening each links as stored in list above
# creating open list for storing all reviews href
all_reviews = []

for l in rev_cli:
    # opening each links as stored in cli_rev open list
    driver.get(l)
    time.sleep(3)
    
    try:
        # finding links from all webpage by xpath
        all_rev = driver.find_element_by_xpath("//div[@id = 'reviews-medley-footer']/div[2]/a")
        all_reviews.append(all_rev.get_attribute('href'))
        # if all reviews link is not available, it will continue to next page,so avoiding NoSuchElementException 
    except NoSuchElementException as e:
        continue

In [8]:
# Checking length of all reviews link
len(all_reviews)

451

In [9]:
# finally opening all_reviews link and scraping ratings and reviews data from all webpages.
# creating open list to save scraped data
ratings = []
reviews = []

# opening each link to begin the scaping
for m in all_reviews:
    driver.get(m)
    time.sleep(15)
    
    # loop to open link till page 5 and scraping data of reviews and ratings of products
    for n in range(0,5):
        for o in driver.find_elements_by_xpath("//div[@class = 'a-section review aok-relative']/div/div/div[2]/a[1]"):
            ratings.append(o.get_attribute('title').replace(' out of 5 stars',''))
            
        for p in driver.find_elements_by_xpath("//div[@class = 'a-section review aok-relative']/div/div/div[4]/span/span"):
            reviews.append(p.text)
                        
        try:
            next_page = driver.find_element_by_xpath("//ul[@class = 'a-pagination']/li[2]/a")
            driver.get(next_page.get_attribute('href'))
        except NoSuchElementException as e:
            continue

In [13]:
# saving scraped data into dataframe.
search = pd.DataFrame({}) 
search['Ratings'] = ratings[:20442]
search['Full_review'] = reviews[:20442]
search

,Ratings,Full_review
0,1.0,Bought the Mi Ultra laptop during the first sa...
1,5.0,Pros:\n1) Build quality.\n2) Display (best I h...
2,1.0,"Overall product is good, but charger is missin..."
3,5.0,Thanks to livetre.com who's reviewing this lap...
4,1.0,You will not get an option to buy extended wa...
...,...,...
20437,4.0,D-link thought to be made from taiwan but its ...
20438,5.0,It did not even connect with PPPoE connection....
20439,4.0,Do not spend your money buying this router. Af...
20440,5.0,"this is not dual band router, this is working ..."


In [14]:
# Saving scraped data in csv format
search.to_csv('Ratings_Prediction_Amazon.csv')